In [2]:
!wget https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark

--2021-10-07 15:25:44--  https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854362 (834K) [text/plain]
Saving to: ‘facebook_combined.txt’

facebook_combined.t 100%[===================>] 834.34K  --.-KB/s    in 0.05s   

2021-10-07 15:25:44 (16.4 MB/s) - ‘facebook_combined.txt’ saved [854362/854362]

--2021-10-07 15:25:59--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224445805 (214M) [application/x-gzip]
Saving to: ‘spark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext


sc = SparkContext("local","ppj")

In [4]:
undirect = sc.textFile('facebook_combined.txt').map(lambda x : x.split(" "))\
.flatMap(lambda x: ((int(x[0]),int(x[1])),(int(x[1]),int(x[0])))).groupByKey().mapValues(list)

threshold = 0.6

In [5]:
A =undirect.map(lambda x: len(x[1])).collect()
B=sorted(range(len(A)),key=lambda x:A[x])
order=sorted(range(len(A)),key=lambda x:B[x])
#sort를 위한 global order

In [6]:
sorted_undirect = undirect.mapValues(lambda x: sorted(x, key=lambda y : order[y]))

In [7]:
ref = sc.broadcast(dict(sorted_undirect.collect()))

In [8]:
def make_invert(item):
    key = item[0]
    arr= item[1]
    result= []
    prefix_len = int(len(ref.value[key]) - threshold*len(ref.value[key]) + 1) #prefix_filtering 
    for i in arr[:prefix_len]:
      result.append((i,key))
    return result

In [9]:
invert = sorted_undirect.flatMap(make_invert).groupByKey().mapValues(list)
invert.count()

2534

In [10]:
def make_pair(item):
    arr = item[1]
    result = []
    for idx,i in enumerate(arr):
        for j in arr[idx+1:]:
          if not len(ref.value[j]) >= threshold*len(ref.value[i]): continue #size filtering |x| >= t * |y| 
          result.append(((i,j),1))
    return result

In [14]:
pairs = invert.flatMap(make_pair).reduceByKey(lambda x,y:x+y) #단순히 중복 제거를 위한
pairs.count()

403188

In [12]:
def compute_jaccard(item):
    x = item[0][0]
    y = item[0][1]

    value = len(list(set(ref.value[x])&set(ref.value[y]))) / len(list(set(ref.value[x])|set(ref.value[y])))
    return threshold <= value

def is_not_friend(item):
    x = item[0][0]
    y = item[0][1]
    target,arr = x,ref.value[y]
    return not (target in arr)

In [13]:
result = pairs.filter(compute_jaccard).filter(is_not_friend).flatMap(lambda x: ((x[0][0],x[0][1]),(x[0][1],x[0][0])))
result.count()

3238